In [1]:
import time
import streamlit as st 
from dotenv import load_dotenv
from processText import extractPdfText, textChunks, vectorDB
from chatWithPDF import get_conversation_chain
from templates import css, bot_template, user_template


import os
import shutil
from PyPDF2 import PdfReader
from langchain.schema import Document
from langchain.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

import chromadb
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [2]:
openai_api_key = ''
CHROMA_PATH = "chroma1"
DATA_PATH = "data/"

In [3]:
def extractPdfText(documents):
    """
    Extracts raw text from the pdfs
    """
    text=""
    for pdf in documents:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text = text + page.extract_text()
            
    return text


def textChunks(text):
    """
    Splits the extracted text to chunks
    """
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
        )
    chunks = text_splitter.split_text(text)
    return chunks


def vectorDB(chunks):
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    #embeddings = openaiEmbeddings(OPEN_API_KEY=OPEN_API_KEY)
    vector_store = FAISS.from_texts(texts=chunks,
                                    embedding=embeddings)
    
    return vector_store


def chromaDB(chunks):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_texts(
        chunks, OpenAIEmbeddings(openai_api_key=openai_api_key), persist_directory=CHROMA_PATH
    )
    db.persist()
   







#########################################

def get_conversation_chain(db):
    llm = ChatOpenAI(openai_api_key=openai_api_key)
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm( 
    llm=llm,
    retriever = db.as_retriever(),
    memory = memory
    )

    return conversation_chain

<h2> 1 time - FAISS

In [4]:
from PyPDF2 import PdfReader

reader = PdfReader("data/Data_Scienstist.pdf")


In [5]:
page = reader.pages[0]
text = page.extract_text()

In [6]:
chunks = textChunks(text)

In [7]:
embeddingsDB = vectorDB(chunks)

In [8]:
model = get_conversation_chain(embeddingsDB)

In [9]:
model({'question': 'What is Spyros job'})

C:\Anaconda3\envs\udacity\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'question': 'What is Spyros job',
 'chat_history': [HumanMessage(content='What is Spyros job'),
  AIMessage(content='Spyros is a Data Scientist.')],
 'answer': 'Spyros is a Data Scientist.'}

<h2> save load - ChromaDB

In [11]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [23]:
chromadb = Chroma.from_documents(documents=documents, 
                                 embedding= OpenAIEmbeddings(openai_api_key=openai_api_key),
                                 persist_directory='db')

In [25]:
chromadb.persist()
chromadb = None

In [28]:
vectordb = Chroma(persist_directory='db', 
                  embedding_function=OpenAIEmbeddings(openai_api_key=openai_api_key))

In [29]:
# https://www.youtube.com/watch?v=cFCGUjc33aU